# Daily Bias detection by ChatGPT

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
csv_file = str(current_dir.parent) + '/VN30F1M/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [4]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765
...,...,...,...,...,...
2025-02-25 14:15:00,1358.4,1360.8,1358.2,1360.5,6708
2025-02-25 14:20:00,1360.3,1362.1,1359.4,1361.0,7025
2025-02-25 14:25:00,1361.0,1362.2,1360.8,1361.7,4896


In [5]:
df = data[(data.index.hour != 14) & (data.index.minute != 30) & (data.index.minute != 45)]

In [7]:
# Calculate Daily Change (Close - Previous Close)
df['Prev_Close'] = df['Close'].shift(1)
df['Daily_Change'] = df['Close'] - df['Prev_Close']

# Define Bias:
# 🔹 Bullish if price makes a higher high & close > open
# 🔹 Bearish if price makes a lower low & close < open
df['Higher_High'] = df['High'] > df['High'].shift(1)
df['Lower_Low'] = df['Low'] < df['Low'].shift(1)
df['Bullish_Close'] = df['Close'] > df['Open']
df['Bearish_Close'] = df['Close'] < df['Open']

# Determine Bias Based on Market Structure
conditions = [
    (df['Higher_High'] & df['Bullish_Close']),
    (df['Lower_Low'] & df['Bearish_Close'])
]
choices = ['Bullish', 'Bearish']
df['Daily_Bias'] = np.select(conditions, choices, default='Neutral')

In [10]:
_9am_df = df[(df.index.hour == 9) & (df.index.minute == 0)]

In [11]:

# Display Last 10 Days
print(_9am_df[['Open', 'High', 'Low', 'Close', 'Daily_Bias']].tail(10))

                       Open    High     Low   Close Daily_Bias
Date                                                          
2025-02-12 09:00:00  1337.4  1340.2  1337.4  1340.2    Bullish
2025-02-13 09:00:00  1331.9  1331.9  1330.2  1331.0    Bearish
2025-02-14 09:00:00  1345.2  1348.9  1345.2  1347.6    Bullish
2025-02-17 09:00:00  1342.0  1342.0  1339.5  1339.9    Bearish
2025-02-18 09:00:00  1335.3  1335.3  1332.8  1333.1    Bearish
2025-02-19 09:00:00  1337.3  1339.1  1337.3  1338.6    Bullish
2025-02-20 09:00:00  1344.7  1347.5  1344.7  1347.4    Bullish
2025-02-21 09:00:00  1345.0  1346.4  1344.7  1346.2    Neutral
2025-02-24 09:00:00  1350.1  1352.2  1350.1  1352.1    Neutral
2025-02-25 09:00:00  1367.0  1367.0  1365.6  1366.0    Neutral
